In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = 'test-281700'
!gcloud config set project {project_id}

#download model..restarted runtime
bucket_name = 'spotify_asr_dataset'
!gsutil -m cp -r gs://{bucket_name}/t5-model-66242.zip /content/
!unzip t5-model-66242.zip

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Copying gs://spotify_asr_dataset/t5-model-66242.zip...
| [1/1 files][789.1 MiB/789.1 MiB] 100% Done  34.5 MiB/s ETA 00:00:00           
Operation completed over 1 objects/789.1 MiB.                                    
Archive:  t5-model-66242.zip
   creating: t5-model-66242/
  inflating: t5-model-66242/spiece.model  
  inflating: t5-model-66242/special_tokens_map.json  
  inflating: t5-model-66242/config.json  
  inflating: t5-model-66242/tokenizer_config.json  
  inflating: t5-model-66242/pytorch_model.bin  


In [ ]:
!pip install transformers
!pip install rouge
!pip install neuralcoref==4.0
!pip install spacy==2.1.0 #notebook crashes on other versions
!python -m spacy download en
!pip install bert-extractive-summarizer

     |████████████████████████████████| 778kB 5.5MB/s 
     |████████████████████████████████| 1.1MB 16.6MB/s 
     |████████████████████████████████| 3.0MB 40.8MB/s 
     |████████████████████████████████| 890kB 55.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=2d673d2678eebd5ea17d675ad9b906838e1d8c6678cca0eba8c933207b2e1868
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 296kB 4.6MB/s 
     |████████████████████████████████| 27.7MB 111kB/s 
     |████████████████████████████████| 2.1MB 48.1MB/s 
     |████████████████████████████████| 3.2MB 49.9MB/s 
     |████████████████████████████████| 92kB 13.1MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.0 which is incompatible.
  Found existing installation: preshed 3.0.2
    Uninstalling preshed-3.0.2:
      Successfu

In [ ]:
import pandas as pd
reference_results = pd.read_csv('150_gold.csv')
reference_results.head(1)

,episode_id,episode_name,episode_description,transcript,textrank_summary,lexrank_summary,lsa_summary,semi_supervised,supervised,extractive,t5_abstractive,first_five
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Recruiting Secrets From a MLM Recruiting Monst...,If you want to start mastering recruiting whic...,Hello everybody. What's going on in this Jess...,Let's download these stories about whoever I d...,You can do whatever networking events and I'm ...,We're going to talk about some powerful recrui...,"You cannot scummy hashtag wisely, or the peop...",In this episode I talk about some powerful re...,"You cannot scummy hashtag wisely, or the peopl...",--- This episode is sponsored by Anchor: The ...,Hello everybody. What's going on in this Jess...


In [ ]:
transcripts = reference_results[['episode_id', 'transcript', 'episode_description']]
transcripts.head(2)

,episode_id,transcript,episode_description
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Hello everybody. What's going on in this Jess...,If you want to start mastering recruiting whic...
1,spotify:episode:0CExTNH4LFqp1ec1mhTd4I,"Ladies, have you been molested don't be a vic...",Don't be a silent victim of crime. a parody fr...


In [ ]:
from summarizer import Summarizer
from summarizer.coreference_handler import CoreferenceHandler
from transformers import *

custom_config = AutoConfig.from_pretrained("SpanBERT/spanbert-base-cased")
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained("SpanBERT/spanbert-base-cased")
custom_model = AutoModel.from_pretrained("SpanBERT/spanbert-base-cased", config=custom_config)

spanbert_model = Summarizer(custom_model = custom_model, custom_tokenizer=custom_tokenizer)

def spanBert(transcript, num_sentences):
  result = spanbert_model(transcript, min_length=60, num_sentences=num_sentences)
  return ''.join(result)

100%|██████████| 40155833/40155833 [00:01<00:00, 25488643.62B/s]


You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#t5
tokenizer = T5Tokenizer.from_pretrained('/content/t5-model-66242')
model = T5ForConditionalGeneration.from_pretrained('/content/t5-model-66242')


def t5_inference(transcript, threshold):
  t5_form = 'summarize: ' + transcript
  tokenized_text = tokenizer.encode(t5_form, return_tensors="pt")
  if len(tokenized_text[0]) > threshold:
    #run out of RAM/crashes on large number of tokens
    revised_text = sent_tokenize(t5_form)
    length = len(revised_text)
    final_text = revised_text[:int(length/2)] #maybe they talk about content in the first half? find proof
    text = ' '.join(final_text)
    return t5_inference(text)
  summary_ids = model.generate(tokenized_text, max_length=150, num_beams=2, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return output

In [ ]:
import string

def clean_up(text):
  head, _ , _ = text.partition(' ---')
  head = head.strip()
  first_letter = head[0].capitalize()
  head = first_letter + head[1:]
  if head[-1] in string.punctuation:
    pass
  else:
    head += '.'
  return head

In [ ]:
def spanbert_t5(transcript, num_sentences, threshold):
  important_sentences = spanBert(transcript, num_sentences)
  summary = t5_inference(important_sentences, threshold)
  summary = clean_up(summary)
  return summary

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def firstN(transcript, threshold):
  sentences = sent_tokenize(transcript)
  return ' '.join(sentences[:threshold])

def firstN_t5(transcript, limit, threshold):
  sentences = firstN(transcript, limit)
  summary = t5_inference(sentences, threshold)
  summary = clean_up(summary)
  return summary

In [ ]:
transcripts['spanBert_t5'] = transcripts.apply(lambda row: spanbert_t5(row['transcript'], 15, 7000), axis=1)
transcripts['firstN_t5'] = transcripts.apply(lambda row: firstN_t5(row['transcript'], 15, 7000), axis=1)
transcripts.to_csv('ensemble_models.csv')
!gsutil -m cp -r /content/ensemble_models.csv gs://{bucket_name}/

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Copying file:///content/ensemble_models.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/3.9 MiB.                                      


In [ ]:
transcripts.iloc[120]['spanBert_t5']

'Today\'s episode we will spell "incomprehensible" and "imitate".'

In [ ]:
transcripts.iloc[120]['firstN_t5']

"Today's episode we will continue our fourth part of week 2 with 4 new words!"

In [ ]:
from rouge import Rouge
rouge = Rouge()
res = rouge.get_scores(transcripts['episode_description'], transcripts['spanBert_t5'], avg=True)
res

{'rouge-1': {'f': 0.1562961161765013,
  'p': 0.17011666602777498,
  'r': 0.2181239527063148},
 'rouge-2': {'f': 0.02877388261910519,
  'p': 0.03231056586118928,
  'r': 0.03715643759980436},
 'rouge-l': {'f': 0.14204551041521832,
  'p': 0.14927195349917105,
  'r': 0.19052653977980027}}

In [ ]:
res = rouge.get_scores(transcripts['episode_description'], transcripts['firstN_t5'], avg=True)
res

{'rouge-1': {'f': 0.16053411596466036,
  'p': 0.18168437658925504,
  'r': 0.23560462583088315},
 'rouge-2': {'f': 0.03896617068237746,
  'p': 0.04890130481722029,
  'r': 0.054619504823355654},
 'rouge-l': {'f': 0.14791806520235593,
  'p': 0.1639100584495298,
  'r': 0.20579340658087347}}